In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('../', 'people')))

In [2]:
from family_graph import FamilyGraph
from utilities import DataGenerator, AgeGroup
import random

generator = DataGenerator()
graphs: list[FamilyGraph] = []

oldest_groups = list(AgeGroup)[-5:]
N = 5

for i in range(N):
    graphs.append(FamilyGraph(generator, 15, random.choice(oldest_groups)))

for i, graph in enumerate(graphs):
    graph.write_to_csv(filename="families.csv", append=i)

[DataGenerator]: Initializing data generator
[DataGenerator]: Data are ready
[FamilyGraph]: Generation started
[FamilyGraph]: Graph generated
[FamilyGraph]: Generation started
[FamilyGraph]: Graph generated
[FamilyGraph]: Generation started
[FamilyGraph]: Graph generated
[FamilyGraph]: Generation started
[FamilyGraph]: Graph generated
[FamilyGraph]: Generation started
[FamilyGraph]: Graph generated
[FamilyGraph]: Written 443 lines to families.csv
[FamilyGraph]: Written 402 lines to families.csv
[FamilyGraph]: Written 399 lines to families.csv
[FamilyGraph]: Written 532 lines to families.csv
[FamilyGraph]: Written 411 lines to families.csv


In [3]:
import random

class School:
    def __init__(self, name, category, city):
        self.name = name
        self.category = category
        self.city = city
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name

class SchoolGenerator:
    def __init__(self, people):
        self.people = self._filter_scholars(self._flatten(people))
        self.schools = self._generate_school()
        self.attended_schools = self._assign_school()
        
    def _flatten(self, lst):
        return [item for sublist in lst for subsublist in sublist for item in subsublist]
    
    def _filter_scholars(self, lst):
        return [p for p in lst if AgeGroup.MIDDLE_CHILD <= p.age.group <= AgeGroup.LATE_YOUNG_ADULT]
    
    def _generate_school(self):
        cities = {p.city for p in self.people}
        categories = ["Scuola Elementare", "Scuola Media", "Scuola Superiore", "Università"]
        templates = ["Scuola Elementare di {0}", "Scuola Media di {0}", "Scuola Superiore di {0}", "Università di {0}"]

        schools = []
        for city in cities:
            for category, template in zip(categories, templates):
                school_name = template.format(city.name)
                schools.append(School(school_name, category, city.name))
        return schools
    
    def _assign_school(self):
        attended = {}
        for p in self.people:
            school = None
            city_schools = [s for s in self.schools if s.city == p.city.name]
            
            if p.age.group == AgeGroup.MIDDLE_CHILD:
                school = next((s for s in city_schools if s.category == "Scuola Elementare"), None)
            elif p.age.group == AgeGroup.LATE_CHILD:
                school = next((s for s in city_schools if s.category == "Scuola Media"), None)
            elif AgeGroup.EARLY_TEEN <= p.age.group <= AgeGroup.LATE_TEEN:
                school = next((s for s in city_schools if s.category == "Scuola Superiore"), None)
            else:
                university = next((s for s in city_schools if s.category == "Università"), None)
                if university:
                    if p.age.group == AgeGroup.EARLY_YOUTH and random.random() <= 0.7:
                        school = university
                    elif p.age.group == AgeGroup.LATE_YOUTH and random.random() <= 0.5:
                        school = university
                    elif p.age.group == AgeGroup.EARLY_YOUNG_ADULT and random.random() <= 0.3:
                        school = university
                    elif random.random() <= 0.1:
                        school = university
                        
            if school:
                attended[school] = attended.get(school, []) + [p.cf]
        return attended
    
    def write_to_csv(self, filename="schools.csv"):
        c = 0
        with open(filename, "w", encoding='utf-8') as f:
            f.write("school,category,city\n")
            for k, v in self.attended_schools.items():
                f.write(f"{k.name},{k.category},{k.city}\n")
                c += 1
        f.close()
        print(f"{c} schools written to {filename}")
    
    def write_attendance_to_csv(self, filename="school_attendance.csv"):
        c = 0
        with open(filename, "w", encoding='utf-8') as f:
            f.write("school,cf\n")
            for k, v in self.attended_schools.items():
                for cf in v:
                    f.write(f"{k.name},{cf}\n")
                    c += 1
        f.close()
        print(f"{c} attendances written to {filename}")


In [4]:
listona = [graph.levels for graph in graphs]
s = SchoolGenerator(listona)

In [5]:
s.write_to_csv()
s.write_attendance_to_csv()

114 schools written to schools.csv
773 attendances written to school_attendance.csv
